In [12]:
import numpy as np
A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = np.array([[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]], dtype=np.int8)
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 1, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)
F = np.array([  # PassFs validation
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [1, 2, 2, 1], 
            [2, 1, 1, 2]], dtype=np.int8)
# F = np.array([  # Passes validation
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1], 
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1]], dtype=np.int8)


GRID = C.copy()
dim = len(GRID)
dim_2 = dim // 2

## Solver

In [4]:
DIGITS = set(range(dim))
FLIP = np.array([0, 2, 1])
GRID = C.copy()

def subsequent(rows, full: set):
    """Checks for subsequent 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False

    for r, row in enumerate(rows):
        if r in full:
            continue
        
        for i in range(len(row)-1):
            if row[i] == row[i+1] != 0:
                if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                    rows[r, i+2] = FLIP[row[i]]
                    changed = True
                    if np.all(row != 0):
                        full.add(r)

                if i > 0 and rows[r, i-1] == 0: # if prev is empty
                    rows[r, i-1] = FLIP[row[i]]
                    changed = True
                    if np.all(row != 0):
                        full.add(r)
                
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True
                
                    if np.all(row != 0):
                        full.add(r)

    return changed

def equal_count(rows, full: set):
    """Checks for equal count of 1s and 2s.
    param rows: 2D array
    param full: set of rows or cols that are already full
    Returns True if any changes are made."""
    changed = False
    for r in (DIGITS - full):
        row = rows[r]
        # print(row)
        if np.sum(row == 1) == dim_2:
            rows[r, np.where(row == 0)[0]] = 2
            full.add(r)
            changed = True

        elif np.sum(row == 2) == dim_2:
            rows[r, np.where(row == 0)[0]] = 1
            full.add(r)
            changed = True
    
    return changed

def unique(rows, full: set):
    """Checks for unique rows and cols.
    Returns True if any changes are made."""
    changed = False

    # Find rows containing exacty two gaps
    gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
    candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
    keks = rows[list(full)]  # [[2 1 1 2]]

    # Compare each candidate with other complete rows
    for i, candy in enumerate(candidates):
        # Find index of colored tiles
        idx = np.where(candy != 0)[0]  # [0 1]
        for kek in keks:
            if np.array_equal(kek[idx], candy[idx]):
                # Replace the two gaps with complementary colors
                gaps = np.where(candy == 0)[0]  # [2, 3]
                rows[gapped_rows_indx[i], gaps] = FLIP[kek[gaps]]

                full.add(gapped_rows_indx[i])
                changed = True
    
    return changed

def solve(grid):
    """Solves grid in place.
    Assumes valid grid."""
    full_rows, full_cols = set(), set()
    while True:
        if subsequent(grid, full_rows):
            continue
        if subsequent(grid.T, full_cols):
            continue
        if equal_count(grid, full_rows):
            continue
        if equal_count(grid.T, full_cols):
            continue
        if unique(grid, full_rows):
            continue
        if unique(grid.T, full_cols):
            continue
        break

solve(GRID)


## Save and load

In [ ]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


## Generator

In [2]:
def pure_random(n=4):
    """Randomly fills the grid with 1s and 2s."""
    return np.random.randint(0, 3, size=(n, n))

kek = pure_random(6)
print(kek)    

[[0 0 0 2 0 0]
 [0 1 1 2 2 1]
 [0 0 2 2 2 1]
 [2 2 2 0 0 2]
 [2 2 1 2 2 1]
 [1 1 2 0 2 2]]


In [14]:
def valid_action(grid, pos, color):
    """Check that placing color at pos is valid."""
    r, c = pos
    if grid[r, c] != 0:  # Neccessary?
        return False

    # Equal count
    if np.sum(grid[r] == color) + 1 > dim_2 or np.sum(grid[:, c] == color) + 1 > dim_2:
        return False

    # Check gap subsequent
    if 0 < r < dim-1 and grid[r-1, c] == grid[r+1, c] == color:
        return False
    if 0 < c < dim-1 and grid[r, c-1] == grid[r, c+1] == color:
        return False

    # Check subsequent
    if r > 1 and grid[r-2, c] == grid[r-1, c] == color:
        return False
    if r < dim-2 and grid[r+2, c] == grid[r+1, c] == color:
        return False
    if c > 1 and grid[r, c-2] == grid[r, c-1] == color:
        return False
    if c < dim-2 and grid[r, c+2] == grid[r, c+1] == color:
        return False
    
    # Check unique
    if len(np.unique(grid, axis=0)) != len(grid):
        return False
    if len(np.unique(grid, axis=1)) != len(grid):
        return False
    
    return True

def valid_grid(grid, verbose=False):
    """Check that grid is valid."""
    dim_2 = len(grid) // 2
    
    # # Equal count
    # if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
    #     return False
    # if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
    #     return False
    
    # # Subsequent (3 in a row)
    # for r in range(dim):
    #     for c in range(dim):
    #         if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
    #             if verbose:
    #                 print("Subsequent")
    #             return False
    #         if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
    #             if verbose:
    #                 print("Subsequent")
    #             return False
    
    # Unique
    full_rows = grid[np.where(np.sum(grid == 0, axis=1) == 0)[0]]
    if len(full_rows) != len(np.unique(full_rows, axis=0)):
        if verbose:
            print("Unique")
        return False
    full_cols = grid.T[np.where(np.sum(grid == 0, axis=0) == 0)[0]]
    if len(full_cols) != len(np.unique(full_cols, axis=0)):
        if verbose:
            print("Unique")
        return False

    return True

for _ in range(2):
    kek = pure_random(6)
    while not valid_grid(kek, True):
        kek = pure_random(6)
    print(kek)
    solve(kek)
    print(kek)
    print()

[[1 0 2 1 0 2]
 [1 1 0 2 2 0]
 [2 2 2 2 2 1]
 [2 2 2 1 2 1]
 [2 2 2 0 1 2]
 [1 2 0 0 0 0]]
[[1 1 2 1 1 2]
 [1 1 2 2 2 2]
 [2 2 2 2 2 1]
 [2 2 2 1 2 1]
 [2 2 2 1 1 2]
 [1 2 0 1 0 0]]

[[0 0 1 1 1 0]
 [2 2 2 0 0 1]
 [1 2 1 0 1 1]
 [1 2 1 1 1 0]
 [2 1 2 0 1 1]
 [1 0 0 0 0 0]]
[[1 2 1 1 1 2]
 [2 2 2 1 2 1]
 [1 2 1 2 1 1]
 [1 2 1 1 1 2]
 [2 1 2 2 1 1]
 [1 2 1 1 2 1]]



In [16]:
GRID = F.copy()
valid_grid(GRID, True)

Unique


False

In [11]:
def follow_consequence(grid, pos, color, full_rows: set, full_cols: set):
    """Returns exhausted grid or None if invalid"""
    grid = grid.copy()    
    full_rows, full_cols = full_rows.copy(), full_cols.copy()

    if not valid_action(grid, pos, color):
        return None
    
    grid[pos] = color
    solve(grid)  # Exhaust the grid
    
    return grid, full_rows, full_cols if valid_grid(grid) else None

def generate_grid():
    new = np.zeros((dim, dim), dtype=np.int8)
    free_tiles = [(i, j) for i in range(dim) for j in range(dim)]
    full_rows, full_cols = set(), set()

    while len(free_tiles) > 0:
        indx = np.random.randint(len(free_tiles))
        pos = free_tiles.pop(indx)
        if new[pos] != 0:
            continue

        color = np.random.randint(1, 3)
        result = follow_consequence(new, pos, color, full_rows, full_cols)
        if result is not None:
            new, full_rows, full_cols = result
        else:
            new[pos] = FLIP[color]
            solve(new)
            assert(valid_grid(new)), "Invalid grid generated :("
            # Follow consequences and assert valid grid, because I think this is guaranteed

            # # Check if pos filled a row or col - TODO: Test it
            # if np.all(new[pos[0]] != 0):
            #     full_rows.add(pos[0])
            # if np.all(new[:, pos[1]] != 0):
            #     full_cols.add(pos[1])
    
    return new


generate_grid()

array([[2, 2, 1, 1],
       [1, 1, 2, 2],
       [2, 1, 1, 2],
       [1, 2, 2, 1]], dtype=int8)